### Carga de modulos

In [ ]:
from keras.models import Sequential, load_model
from keras import layers
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from datetime import datetime as dt
import numpy as np
import pandas as pd
import os

### Carga del dataset

In [ ]:
(X_train, y_train_lab), (X_test, y_test_lab) = mnist.load_data()
X_train = X_train / 255
X_test = X_test / 255

X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)
y_train = to_categorical(y_train_lab)
y_test = to_categorical(y_test_lab)

### ¿Cómo ve una imagen un PC?

In [ ]:
image_0 = (X_train[0, :] * 255).reshape(28, 28).astype(np.int32) 
df_image = pd.DataFrame(image_0)

In [ ]:
df_image

In [ ]:
plt.imshow(image_0, cmap='gray_r')

### Ejemplo de algunas imagenes

In [ ]:
def plot_images_sample(X, y, prepend_text='Original'):
    n_images = 9 
    index = np.random.choice(np.arange(len(X)), size=n_images, replace = False)
    
    X_plot = X[index, : ]
    y_plot = y[index]
    
    fig, axes = plt.subplots(3,3, 
                         figsize=(5,5),
                         sharex=True, sharey=True,
                         subplot_kw=dict(aspect='equal'))
    
    for i, image in enumerate(X_plot):
    
        row = i//3 
        col = i%3  

        ax = axes[row, col]
        img_plot = np.reshape(image, (28,28))
        ax.imshow(img_plot, cmap='gray_r')
        ax.set_title('{} label: {}'.format(prepend_text, y_plot[i]))
        ax.set_xbound([0,28])
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_images_sample(X_train, y_train_lab)

### Definición de modelo (LeNet5) con un cambiós sobre la primera convolución

<img src="LeNet_Original_Image.jpg"> 

In [ ]:
model = Sequential()
model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', 
                        input_shape=(28,28,1), padding='same'))
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=2))
model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
model.add(layers.AveragePooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(units=120, activation='relu'))
model.add(layers.Dense(units=84, activation='relu'))
model.add(layers.Dense(units=10, activation = 'softmax'))

### Resumen del modelo

In [ ]:
model.summary()

### Logs para tensorboard

In [ ]:
all_logs_path = os.path.join(os.getcwd(), 'logs')
if not os.path.exists(all_logs_path):
    os.mkdir(all_logs_path)
    
now = dt.now()
str_now = now.strftime('%Y-%m-%d_%H:%M:%S')
log_dir = os.path.join(all_logs_path, 'MNist_Keras{}'.format(str_now))
tensorboard = TensorBoard(log_dir=log_dir)

### Entrenamiento del modelo

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy'])

start = dt.now()
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          callbacks=[tensorboard], epochs=4, batch_size=64)
print(dt.now() - start)

### Evaluación de algunas predicciones

In [ ]:
pred = model.predict(X_test)
pred_lab = np.array([np.argmax(x) for x in pred])
plot_images_sample(X_test, pred_lab, 'Predicted')

### Guardar y cargar modelo

In [ ]:
# Guardar modelo
path_saved_models = os.path.join(os.getcwd(), 'saved_models')
model.save(os.path.join(path_saved_models, 'MNist_Keras.h5'))

# Cargar modelo
model_load = load_model(os.path.join(path_saved_models, 'MNist_Keras.h5'))

In [ ]:
model_load.summary()

### Predicciones modelo cargado

In [ ]:
pred = model_load.predict(X_test)
pred_lab = np.array([np.argmax(x) for x in pred])
plot_images_sample(X_test, pred_lab, 'Predicted')